--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://github.com/giakou4)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# YOLOv3, YOLOv4 by [Darknet](https://github.com/AlexeyAB/darknet)
Read [this](https://github.com/AlexeyAB/darknet) before.


# 1 Clone and Build **```darknet```**

In [ ]:
%%capture

# Install roboflow
!pip install -q roboflow

# Clone darknet repository
!git clone https://github.com/AlexeyAB/darknet

# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

# Make darknet (build)
!make

# 2 Define Helper Function

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

# 3 Dataset

Download data from Robolflow in ```YOLO Darknet``` format

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="7EOGxE0Zhj6CaawbEz1J")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("darknet")

# 4 Download pre-trained weights for the convolutional layers

In [ ]:
!wget http://pjreddie.com/media/files/darknet53.conv.74
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# 5 Configuring Files for Training

## 5.1 Set up training file directories for custom dataset

1. Create a directory **```darknet/data/obj```** where to save all training and validation images and txt files.

2. Create a new file within a code or text editor called **```darknet/data/obj.data```** where you will save the classes, train txt, valid txt, names and backup. This backup path is where we will save the weights to of our model throughout training. Create a backup folder in your google drive and put its correct path in this file.

3. Create a new file within a code or text editor called **```darknet/data/obj.names```** where you will have one class name per line.

4. Generate **```darknet/data/train.txt```** and **```darknet/data/test.txt```**. The last configuration files needed before we can begin to train our custom detector are the train.txt and test.txt files which hold the relative paths to all our training images and valdidation images.

> _Do not forget to manually change the number of classes_

In [ ]:
%cd /content/darknet/

# Copy names to "darknet/data/obj.names"
%cp {dataset.location}/train/_darknet.labels data/obj.names
%mkdir data/obj

# Copy image and labels to "darknet/data/obj"
%cp {dataset.location}/train/*.jpg data/obj/
%cp {dataset.location}/valid/*.jpg data/obj/

%cp {dataset.location}/train/*.txt data/obj/
%cp {dataset.location}/valid/*.txt data/obj/

# Write "darknet/data/obj.data"
with open('data/obj.data', 'w') as out:
  out.write('classes = 3\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

# Copy "darknet/data/train.txt" images to darknet/data/obj
import os
with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir(dataset.location + '/train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

# Write the "darknet/data/test.txt" file (just the image list)
import os
with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir(dataset.location + '/valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

## 5.2 Calculate anchors

In [ ]:
width = 416 #@param {type:"integer"}
height = 416 #@param {type:"integer"}

In [ ]:
#!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width {width} -height {height}

## 5.3 Cfg File

Copy over the yolov3.cfg to edit. Rename to **```custom-yolov3-detector.cfg```**. Copy over the yolov4.cfg to edit. Rename to **```custom-yolov4-detector.cfg```**. Check this [link](https://www.youtube.com/watch?v=zJDUhGL26iU&t=300s&ab_channel=TheAIGuy) on how to edit. Basically one must change:

* **```batch```** to 64
* **```subdivisions```** to 16. If failed to 32. If failed to 64
* **```max_batches```** to 2000 times the ```num_classes``` (minimum 6000)
* **```steps```** to 80% and 90% of ```max_batches```
* **```width```** to 416. **```height```** to 416 or any multiple of 32. 416 is standard, 608 can improve results but slow inference/training. Usually set both to 416 or both to 608.
* the number of classes on each of the 3 ```[yolo]``` section and change the covolution layer's filters above to **```filters=(num_classes+5)x3```**
* (opt.) the **```random```** on each of the 3 ```[yolo]``` section to 1 (to resize) - will lower accuracy but save memory
* (opt.) change **```anchors```** based on the above output script

In [ ]:
!cp cfg/yolov3.cfg /content/darknet/cfg/custom-yolov3-detector.cfg
!cp cfg/yolov4.cfg /content/darknet/cfg/custom-yolov4-detector.cfg
assert False

# 6 YOLOv3

In [ ]:
!./darknet detector train data/obj.data cfg/custom-yolov3-detector.cfg darknet53.conv.74 -dont_show -map
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/custom-yolov3-detector.cfg backup/custom-yolov3-detector_best.weights -points 101

In [ ]:
for t in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    !darknet.exe detector map data/obj.data cfg/custom-yolov3-detector.cfg backup/custom-yolov3-detector_best.weights -iou_thresh {t}

# 7 YOLOv4

In [ ]:
!./darknet detector train data/obj.data cfg/custom-yolov4-detector.cfg yolov4.conv.137 -dont_show -map
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_best.weights -points 101

In [ ]:
for t in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    !darknet.exe detector map data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_best.weights -iou_thresh {t}